In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('C:/Users/연구실/연구실 프로젝트/data/전처리최종이었음좋겠다.csv')
data.head()

,yearmonth,Ticker,한국표준산업분류코드10차(중분류),수익률 (1개월)(%),"베타 (D,1Yr)",PER(배),PSR(배),chmom,mom36m,zerotrade,...,원/달러환율,금($/OZ),뉴스심리지수,KOSPI dp,KOSPI PER,KOSPI PBR,CD Rate,Term spread,Default Spread,VKOSPI 200
0,2004-01,A005930,26,1.87,0.756757,-0.133333,0.670270,-0.084685,0.812613,-1.0,...,1172.1,402.45,100.0,1.99,10.72,1.07,4.26,1.16,4.32,21.23
1,2004-02,A005930,26,4.95,0.764388,-0.124101,0.688849,-0.383094,0.879496,-1.0,...,1171.6,396.15,100.0,1.99,10.94,1.08,4.08,1.35,4.40,19.95
2,2004-03,A005930,26,-0.54,0.861261,-0.072072,0.713514,0.623423,0.846847,-1.0,...,1153.3,426.45,100.0,1.83,11.19,1.11,3.93,1.33,4.48,24.66
3,2004-04,A005930,26,-9.16,0.905626,-0.081670,0.709619,-0.618875,0.809437,-1.0,...,1170.7,386.75,100.0,2.21,16.76,1.10,3.91,1.34,4.47,24.47
4,2004-05,A005930,26,-8.27,0.881081,-0.079279,0.720721,-0.621622,0.881081,-1.0,...,1164.9,395.55,100.0,2.39,15.40,1.02,3.90,1.26,4.47,30.14


In [ ]:
X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', '뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [ ]:
# cross_validate 적용
import warnings
warnings.filterwarnings("ignore")

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import joblib

a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
sse = 0
ya0 = []
tk0 = []
ym0 = []
ya1 = []

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvym = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    Vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    rf_search_space = {
        # 노드를 분할하기 위한 최소한의 샘플 데이터 수 : 작게 설정할수록 분할되는 노드가 많아져서 과적합 가능성 증가 (default=2)
        'min_samples_split' : hp.quniform('min_samples_split', 2, 10, 2), # 8, 20, 4
        'max_depth' : hp.quniform('max_depth', 1, 6, 1)
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        rf = RandomForestRegressor(
            n_estimators = 500,
            min_samples_split = int(search_space['min_samples_split']),
            max_depth = int(search_space['max_depth']),
            random_state = 1 
        )
        
        d = 0
        
        # time series cv
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            rf.fit(tX, ty)
            
            # r2oos 계산         
            y_pred = pd.Series(rf.predict(vX).reshape(-1,)) # sort_values() 사용을 위해 변환
            d = d + sum((vy-y_pred)**2) 

        e = sum(Vy**2)
        r2oos = 1-(d/e)
        new_loss = r2oos*(-1)

        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=rf_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['min_samples_split'] = int(best['min_samples_split'])
    best['max_depth'] = int(best['max_depth'])
    print(best) 
    
    reg0 = RandomForestRegressor(
        n_estimators = 500,
        max_features = "sqrt",
        random_state = 1, 
        **best)
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    joblib.dump(reg0,'RF_bayesian_r2oos_last3_'+str(i)+'.pkl')
    sse = sse + sum((oy-yy0)**2)

oyy = y[ym.str.contains(a[40:84])]
e = sum(oyy**2)
r2oos = 1-(sse/e)
print(r2oos)

RF = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
RF.to_csv('RF_bayesian_r2oos_last3.csv')    

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]


  2%|▊                                    | 23/1000 [8:37:13<366:10:52, 1349.29s/trial, best loss: 0.00891017922858306]
{'max_depth': 1, 'min_samples_split': 4}


 11%|████████▍                                                                   | 1/9 [8:37:55<69:03:25, 31075.73s/it]


  2%|▊                                 | 23/1000 [10:27:53<444:31:29, 1637.96s/trial, best loss: -0.002982099794609838]
{'max_depth': 1, 'min_samples_split': 4}


 22%|████████████████▋                                                          | 2/9 [19:06:37<68:01:37, 34985.39s/it]

...


  2%|▊                                 | 23/1000 [10:03:47<427:28:08, 1575.12s/trial, best loss: -0.012924203232865583]
{'max_depth': 1, 'min_samples_split': 4}


 89%|███████████████████████████████████████████████████████████████████▌        | 8/9 [82:23:05<9:19:15, 33555.87s/it]


  2%|▊                                | 23/1000 [26:28:57<1124:55:57, 4145.10s/trial, best loss: -0.009907220560961938]
{'max_depth': 1, 'min_samples_split': 4}


100%|█████████████████████████████████████████████████████████████████████████████| 9/9 [108:53:42<00:00, 43558.06s/it]


0.011401779139930568


In [ ]:
import pandas as pd
import numpy as np
RF = pd.read_csv("RF_bayesian_r2oos_last3.csv")

In [ ]:
ya0 = pd.Series(RF['real'])
ya1 = pd.Series(RF['predict'])
tk0 = pd.Series(RF['ticker'])
ym0 = pd.Series(RF['yearmonth'])

r1 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())

rr1 = 1
rrf1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rrf1.append(rr1)

In [ ]:
rrf1[-1]

26.198260910174355

In [ ]:
r1 = []
r2 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

In [ ]:
for i in range(9):
    rr1 = 1
    for j in r1[0+i*12:12+i*12]:
        rr1 = rr1*(j+100)/100
    print("year:" ,i, "test 수익률:", rr1)

year: 0 test 수익률: 1.5386697122934894
year: 1 test 수익률: 1.4340331499926062
year: 2 test 수익률: 1.5442446719261795
year: 3 test 수익률: 1.6646971414176477
year: 4 test 수익률: 1.4528980243929441
year: 5 test 수익률: 1.249125129214405
year: 6 test 수익률: 1.2911158573922858
year: 7 test 수익률: 1.190652396884214
year: 8 test 수익률: 1.6554849518821175
